In [1]:
!pip install -q -U autogluon.tabular

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.0 requires aiohttp<4.0.0,>=3.9.2, which is not installed.
s3fs 2024.3.1 requires aiohttp!=4.0.0a0,!=4.0.0a1, which is not installed.
aiobotocore 2.13.0 requires botocore<1.34.107,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


# Imports and Configs

In [2]:
from autogluon.tabular import TabularPredictor, TabularDataset
import pandas as pd
import warnings
import shutil
import pickle
import glob

warnings.filterwarnings('ignore')

In [3]:
TIME_LIMIT = 1800
SEED = 27
TARGET = 'Target'

# Loading and Preprocessing the Datasets

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv', index_col='id')

train.shape, test.shape

((76518, 37), (51012, 36))

In [5]:
original = pd.read_csv('/kaggle/input/students-dropout-and-academic-success/data.csv', delimiter=';')
original = original.rename(columns={'Daytime/evening attendance\t': 'Daytime/evening attendance'})

train = pd.concat([train, original], axis=0, ignore_index=True)

train.shape

(80942, 37)

In [6]:
# Reference: https://www.kaggle.com/code/ambrosm/pss4e6-eda-which-makes-sense
cat_features = [
    'Marital status', 
    'Application mode', 
    'Course',
    'Previous qualification', 
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation"
]
for feature in cat_features:
    for df in [train, test]:
        df[feature] = df[feature].astype('category').cat.codes

In [7]:
train = TabularDataset(train)
test = TabularDataset(test)

# Training

In [8]:
predictor = TabularPredictor(
    label=TARGET,
    eval_metric='accuracy',
    problem_type='multiclass',
    verbosity=2
).fit(
    train_data=train,
    time_limit=TIME_LIMIT,
    presets='best_quality',
    excluded_model_types = ['KNN'],
    save_space=False,
    hyperparameter_tune_kwargs={  
        'num_trials': 25,
        'scheduler' : 'local',
        'searcher'  : 'auto'
    },
    feature_prune_kwargs={'force_prune': True}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240601_062027"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 1800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240601_062027/ds_sub_fit/sub_fit_ho.
2024-06-01 06:20:28,399	INFO util.py:124 -- Outdated packages:
  ipywidgets==7

  0%|          | 0/25 [00:00<?, ?it/s]

Will use sequential fold fitting strategy because import of ray failed. Reason: ray==2.9.0 detected. 2.10.0 <= ray < 2.11.0 is required. You can use pip to install certain version of ray `pip install ray==2.10.0` 
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
No model was trained during hyperparameter tuning NeuralNetFastAI_BAG_L1... Skipping this model.
Hyperparameter tuning model: LightGBMXT_BAG_L1 ... Tuning model for up to 2.5s of the 444.06s of remaining time.


  0%|          | 0/25 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's multi_error: 0.524461
	Stopping HPO to satisfy time limit...
No model was trained during hyperparameter tuning LightGBMXT_BAG_L1... Skipping this model.
Hyperparameter tuning model: LightGBM_BAG_L1 ... Tuning model for up to 2.5s of the 442.43s of remaining time.


...

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
No model was trained during hyperparameter tuning NeuralNetTorch_r19_BAG_L2... Skipping this model.
Hyperparameter tuning model: XGBoost_r95_BAG_L2 ... Tuning model for up to 5.58s of the -134.92s of remaining time.
	No hyperparameter search space specified for XGBoost_r95_BAG_L2. Skipping HPO. Will train one model based on the provided hyperparameters.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 1334, in _hyperparameter_tune
    hpo_executor.validate_search_space(search_space, self.name)
  File "/opt/conda/lib/python3.10/site-packages/autogluon/core/hpo/executors.py", line 512, in validate_search_space
    raise EmptySearchSpace
autogluon.core.hpo.exceptions.EmptySearchSpac

  0%|          | 0/25 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
No model was trained during hyperparameter tuning NeuralNetTorch_r1_BAG_L2... Skipping this model.
Hyperparameter tuning model: NeuralNetTorch_r89_BAG_L2 ... Tuning model for up to 5.58s of the -161.8s of remaining time.


  0%|          | 0/25 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
No model was trained during hyperparameter tuning NeuralNetTorch_r89_BAG_L2... Skipping this model.
No models of type <class 'autogluon.tabular.models.lgb.lgb_model.LGBModel'> have finished training in level 2. Skipping feature pruning.
Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -171.16s of remaining time.
	Ensemble Weights: {'ExtraTrees_r178_BAG_L1': 0.16, 'RandomForestEntr_BAG_L1': 0.12, 'XGBoost_r33_BAG_L1': 0.12, 'ExtraTrees_r172_BAG_L1': 0.12, 'ExtraTrees_r126_BAG_L2': 0.12, 'RandomForestGini_BAG_L1': 0.08, 'XGBoost_r34_BAG_L1': 0.08, 'ExtraTreesGini_BAG_L1': 0.04, 'LightGBM_r130_BAG_L1': 0.04, 'XGBoost_r49_BAG_L1': 0.04, 'RandomForest_r166_BAG_L1': 0.04, 'XGBoost_r22_BAG_L1': 0.04}
	0.827	 = Validation score   (accuracy)
	6.04s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime 

In [9]:
oof_predictions = predictor.get_oof_pred_proba()
with open('autogluon_oof_preds.pkl', 'wb') as f:
    pickle.dump(oof_predictions, f)

# Inference

In [10]:
preds = predictor.predict_proba(test)

In [11]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv')
submission[TARGET] = preds.idxmax(axis=1).values
submission.to_csv('submission.csv', index=False)
submission.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled


In [12]:
shutil.rmtree("AutogluonModels")